In [ ]:
# load image and convert to and from NumPy array
from PIL import Image
from numpy import argmax
import numpy as np
import cv2
from PIL import Image

train_pic_path= [os.path.join(train_dir, filename)for filename in train_x]
train_image = []
for i, img_path in enumerate(train_pic_path):
    img_path = img_path + ".png"
    img = Image.open(img_path)
    train_image.append(img)
print(len(train_image))

data_out = [0,1,2,3,4]
label_int =  dict((c, i) for i, c in enumerate(data_out))
int_labe = dict((i, c) for i, c in enumerate(data_out))
interger_encoded = [label_int[char] for char in train_labels]

onehot_encoded = []
for value in interger_encoded:
    labe_n = [0 for _ in range(len(data_out))]
    labe_n[value] = 1
    onehot_encoded.append(labe_n)
train_onehot_labe = onehot_encoded

data_frame_work = pd.DataFrame({"Train_label": train_onehot_labe,
                               "Train_image": train_image,
                               "Triain_id": train_x},
                              columns=["Train_label","Train_image","Triain_id"])

# CNN
## Need to standard image size

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(2, 2), input_shape=[512,512,3], activation= 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(GlobalAveragePooling2D())
model.add(Dense(units=128, activation = 'relu'))
model.add(Dense(units=256, activation = 'relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.00005), metrics=['accuracy'])

# FCN

In [ ]:
# To solve the unstandard size images, find out the highest, widest image, and add 0 to others to make them become same size.By doing so, we still can pass it to fit_generator()
def image_batch(image_group, BATCH_SIZE):
    # get the max image shape
    max_shape = tuple(max(image.shape[x] for image in image_group) for x in range(3))

    # construct an image batch object
    image_batch = np.zeros((BATCH_SIZE,) + max_shape, dtype='float32')

    # copy all images to the upper left part of the image batch object
    for image_index, image in enumerate(image_group):
        image_batch[image_index, :image.shape[0], :image.shape[1], :image.shape[2]] = image

    return image_batch

In [1]:
import tensorflow as tf

def FCN_model(len_classes=5, dropout_rate=0.2):
    
    # Input layer
    input = tf.keras.layers.Input(shape=(None, None, 3)) # because we don't have a standarded so set as None, None to represent width, height

    # A convolution block
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)(input)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # Fully connected layer 1
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Fully connected layer 2
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    model = tf.keras.Model(inputs=input, outputs=predictions)
    print(model.summary())